https://www.kaggle.com/therohk/urban-dictionary-words-dataset/data

<p id="toc"></p>
## Table of Contents

   - <a href="#one">Introduction</a>
   - <a href="#two">Import Libraries</a>
   - <a href="#three">Import Data</a>
   - <a href="#four">Explore Data</a>
   - <a href="#five">Feature Selection</a>
   - <a href="#six">Model Development</a>  
   - <a href="#seven">Model Validation</a>  
   - <a href="#eight">Recommendation</a>
   - <a href="#nine">References</a>

<hr style="width:50%;">
# <a id="one">Introduction</a>
<p><a href="#toc">Jump to top</a></p>

<hr style="width:50%;">
# <a id="two">Import Libraries</a>
<p><a href="#toc">Jump to top</a></p>

In [59]:
import pandas as pd

# For the google API
from google.cloud import storage
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
import six

# for AWS API
import boto3

<hr style="width:50%;">
# <a id="three">Import Data</a>
<p><a href="#toc">Jump to top</a></p>

In [60]:
urban = pd.read_csv('urbandict_word_def.csv', error_bad_lines=False)

b'Skipping line 3697: expected 6 fields, saw 7\nSkipping line 5549: expected 6 fields, saw 7\nSkipping line 7203: expected 6 fields, saw 7\nSkipping line 9771: expected 6 fields, saw 7\nSkipping line 13384: expected 6 fields, saw 7\nSkipping line 20037: expected 6 fields, saw 7\nSkipping line 20153: expected 6 fields, saw 7\nSkipping line 21825: expected 6 fields, saw 8\nSkipping line 23873: expected 6 fields, saw 8\nSkipping line 25305: expected 6 fields, saw 7\nSkipping line 25692: expected 6 fields, saw 7\nSkipping line 25832: expected 6 fields, saw 7\nSkipping line 31046: expected 6 fields, saw 7\nSkipping line 35583: expected 6 fields, saw 7\nSkipping line 36117: expected 6 fields, saw 8\nSkipping line 36168: expected 6 fields, saw 7\nSkipping line 40252: expected 6 fields, saw 7\nSkipping line 40799: expected 6 fields, saw 7\nSkipping line 42051: expected 6 fields, saw 7\nSkipping line 46657: expected 6 fields, saw 7\nSkipping line 48635: expected 6 fields, saw 7\nSkipping line 4

b'Skipping line 788052: expected 6 fields, saw 7\nSkipping line 789962: expected 6 fields, saw 8\nSkipping line 808965: expected 6 fields, saw 7\nSkipping line 842268: expected 6 fields, saw 7\nSkipping line 847558: expected 6 fields, saw 8\nSkipping line 849857: expected 6 fields, saw 8\nSkipping line 850708: expected 6 fields, saw 7\nSkipping line 851928: expected 6 fields, saw 7\nSkipping line 852184: expected 6 fields, saw 7\nSkipping line 856597: expected 6 fields, saw 7\nSkipping line 876015: expected 6 fields, saw 7\nSkipping line 882608: expected 6 fields, saw 8\nSkipping line 891297: expected 6 fields, saw 7\nSkipping line 895388: expected 6 fields, saw 7\nSkipping line 896959: expected 6 fields, saw 7\nSkipping line 909669: expected 6 fields, saw 7\n'
b'Skipping line 917809: expected 6 fields, saw 8\nSkipping line 937053: expected 6 fields, saw 7\nSkipping line 944846: expected 6 fields, saw 7\nSkipping line 967451: expected 6 fields, saw 8\nSkipping line 968802: expected 6 f

b'Skipping line 2373282: expected 6 fields, saw 8\nSkipping line 2440667: expected 6 fields, saw 7\nSkipping line 2460888: expected 6 fields, saw 7\nSkipping line 2473629: expected 6 fields, saw 7\n'
b'Skipping line 2540378: expected 6 fields, saw 7\nSkipping line 2563083: expected 6 fields, saw 7\nSkipping line 2586628: expected 6 fields, saw 7\nSkipping line 2604742: expected 6 fields, saw 7\n'
/Users/scottvirshup/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [61]:
urban.head()

,word_id,word,up_votes,down_votes,author,definition
0,9.0,yayeeyay,19,27,dc397b2f,"affirmation; suggestion of encouragement, appr..."
1,7.0,Janky,296,255,dc397b2f,Undesirable; less-than optimum.
2,13.0,brutal,12,45,40ece1ef,anything that makes you sweat
3,14.0,skanky,9,48,485e4db7,"Anything of or pertaining to a $10,000 hooker."
4,22.0,hecka,8,18,b9dcf126,see synonyms at hella.


In [62]:
urban.shape

(2606171, 6)

In [63]:
urban.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2606171 entries, 0 to 2606170
Data columns (total 6 columns):
word_id       float64
word          object
up_votes      object
down_votes    object
author        object
definition    object
dtypes: float64(1), object(5)
memory usage: 119.3+ MB


In [64]:
urban['up_votes'] = urban['up_votes'].fillna(0)
urban['down_votes'] = urban['down_votes'].fillna(0)

urban = urban.dropna()

In [65]:
urban.isnull().sum()

word_id       0
word          0
up_votes      0
down_votes    0
author        0
definition    0
dtype: int64

In [66]:
print (urban[pd.to_numeric(urban['up_votes'], errors='coerce').isnull()])

           word_id                                           word   up_votes  \
1620246  4472044.0  THINGAMAJIGGER'',12,7,1483fd5e,Anything thing  component   

        down_votes author                                         definition  
1620246       part  piece  item etc....that you cannot remember the name ...  


In [67]:
# identify rows where numeric fails, and drop those rows based on their index
#print (urban[pd.to_numeric(urban['up_votes'], errors='coerce').isnull()])


# 750414 is the n-word. dropping it out of principle
urban = urban.drop([1620246,750414])

# turn up votes and down votes into integers
urban['up_votes'] = urban['up_votes'].astype('int64')
urban['down_votes'] = urban['down_votes'].astype('int64')

<hr style="width:50%;">
# <a id="four">Explore the Data</a>
<p><a href="#toc">Jump to top</a></p>

In [68]:
urban.shape

(2605912, 6)

In [69]:
# create a "total votes" column
urban['total_votes'] = urban['up_votes'] + urban['down_votes']


In [70]:
urban['total_votes'].describe()

count    2.605912e+06
mean     1.076029e+02
std      8.079693e+02
min     -5.400000e+01
25%      5.000000e+00
50%      1.600000e+01
75%      5.400000e+01
max      3.705830e+05
Name: total_votes, dtype: float64

In [71]:
urban.sample(5)

,word_id,word,up_votes,down_votes,author,definition,total_votes
2490240,8031747.0,C'est le tarif,4,8,10ed8bae,Phrase used to say it's a punishment for a rid...,12
2024542,5766410.0,Before You Exit,53,5,91c95a28,An alternative / pop punk / rock band from Orl...,58
2161679,6181405.0,vajaggle,8,0,47f46b33,1. (usually accidental) snagging or catching o...,8
384271,1023243.0,San Francisco,98,67,3369703,I have found that while traveling the world th...,165
1251479,3395776.0,slugga,9,11,ea5a116e,English slang used mostly in southern Florida ...,20


In [72]:
# top 5 definitions with the most TOTAL VOTES
urban.sort_values(by=['total_votes'], ascending=False).head(5)

,word_id,word,up_votes,down_votes,author,definition,total_votes
2465548,7910405.0,cartossin,270206,100377,c5a1894f,The act of giving up on both physical and ment...,370583
575828,1539760.0,sex,289002,79134,3624fb48,What kind of moron are you that you look up se...,368136
990126,2705928.0,hipster,182625,42696,8a5ef3c3,Hipsters are a subculture of men and wome...,225321
441475,1146634.0,sex,108716,57510,5ed4dfd3,"sex is like math: you add the bed, subtract th...",166226
728,1153.0,sex,100007,64827,6fc00870,"A painful activity in which a man, using the s...",164834


In [73]:
# top 5 definitions with the most UPVOTES
urban.sort_values(by=['up_votes'], ascending=False).head(5)

,word_id,word,up_votes,down_votes,author,definition,total_votes
575828,1539760.0,sex,289002,79134,3624fb48,What kind of moron are you that you look up se...,368136
2465548,7910405.0,cartossin,270206,100377,c5a1894f,The act of giving up on both physical and ment...,370583
990126,2705928.0,hipster,182625,42696,8a5ef3c3,Hipsters are a subculture of men and wome...,225321
441475,1146634.0,sex,108716,57510,5ed4dfd3,"sex is like math: you add the bed, subtract th...",166226
728,1153.0,sex,100007,64827,6fc00870,"A painful activity in which a man, using the s...",164834


In [74]:
# top 5 definitions with the most DOWNVOTES
urban.sort_values(by=['down_votes'], ascending=False).head(5)

,word_id,word,up_votes,down_votes,author,definition,total_votes
2465548,7910405.0,cartossin,270206,100377,c5a1894f,The act of giving up on both physical and ment...,370583
575828,1539760.0,sex,289002,79134,3624fb48,What kind of moron are you that you look up se...,368136
728,1153.0,sex,100007,64827,6fc00870,"A painful activity in which a man, using the s...",164834
441475,1146634.0,sex,108716,57510,5ed4dfd3,"sex is like math: you add the bed, subtract th...",166226
523014,1384410.0,sex,69475,44372,11800021,Where a man inserts his erect penis into a wom...,113847


In [88]:
# top 5 definitions with the BEST RATIO (upvotes / downvotes)
urban['ratio'] = urban['up_votes'] / urban['down_votes']
        
# create a subset so the ratio isn't as influenced by small numbers
urban_ratio_subset = urban[urban['total_votes'] > 1000]
urban_ratio_subset = urban_ratio_subset[urban_ratio_subset['down_votes'] >= 0]
#urban_ratio_subset['down_votes'] = urban_ratio_subset[urban_ratio_subset['down_votes'] < 0] = 0

best_ratio = urban_ratio_subset.sort_values(by=['ratio'], ascending=False).head(10)
best_ratio

,word_id,word,up_votes,down_votes,author,definition,total_votes,ratio
2325157,7070316.0,Taffanaut,1585,0,b82a7b78,Person born in Wales (UK) who has travelled in...,1585,inf
2429196,7731747.0,yofam,2067,0,ae5f5426,Yofam is a greeting from one person to another...,2067,inf
2354713,7219861.0,Thanksgivukkah,3860,2,69586308,Thanksgivukkah is the name given to the conver...,3862,1930.000000
2426255,7705693.0,seachellefly,4140,6,da0fa52a,"A cheating, conniving, lying person of the fem...",4146,690.000000
2464261,7905086.0,Wisdom Bomb,1106,2,e8fceb4e,Sage advice given.,1108,553.000000
2490678,8033884.0,rateboost,1028,2,70d8e7c4,To dishonestly boost your rates for personal g...,1030,514.000000
2228315,6466831.0,transpants,4408,9,e8fceb4e,Pants designed for the female figure that add ...,4417,489.777778
2389282,7431915.0,lady giblets,1773,4,e8fceb4e,Female reproductive organs.,1777,443.250000
2286211,6851605.0,SigEp Saddle,1277,3,72639e78,The name given to a sorostitute/slam-piece tha...,1280,425.666667
2236827,6523029.0,Social Securitits,1583,4,3682453e,Women over the age of 55 who refuse to wear a ...,1587,395.750000


In [89]:
# top 5 definitions with the WORST RATIO (upvotes / downvotes)
worst_ratio = urban_ratio_subset.sort_values(by=['ratio'], ascending=True).head(10)
worst_ratio

,word_id,word,up_votes,down_votes,author,definition,total_votes,ratio
1297288,3595236.0,Chuck Norris,2,3225,2a4b26be,"The one Supreme Being, the creator and ruler o...",3227,0.000620
1011586,2758972.0,chuck norris,2,3203,cd0de555,Chuck Norris is allegedly the only man to ever...,3205,0.000624
1600263,4415866.0,cunt,2,2905,623dffdd,I word that you should never fucking say you s...,2907,0.000688
363848,966591.0,cudoos,3,3916,b6a386ef,basically just another word for [cool];;[wicke...,3919,0.000766
2073402,5911541.0,Chuck Norris,3,3198,244bcc15,A man who has 4 biographies written simply on ...,3201,0.000938
2143312,6124928.0,Chuck-Norris,3,3181,c746abf7,A Super Human Who Can Do Anything...!,3184,0.000943
1339648,3718026.0,H-O-L-L-A-N-D,3,3068,64807037,A old fashioned acronymn for:;;Here;;Our;;Love...,3071,0.000978
1032912,2804865.0,shadana O'connor,8,6851,c78622,biggest fuckin slut in the world!,6859,0.001168
2002270,5711040.0,chuck norris,4,3210,89d42301,the sum of everything awesome and gnarley,3214,0.001246
731871,1954079.0,forgoogle,2,1542,d4ca24e4,to get rid of an unpleasant fact by suppressin...,1544,0.001297


<hr style="width:50%;">
# <a id="five">First API - Google</a>
<p><a href="#toc">Jump to top</a></p>

* https://cloud.google.com/natural-language/docs/analyzing-entities#language-entities-string-python

In order to make this work, you have to launch jupyter notebook with an authentication key. The steps to do so are outlined in the "Using Python API" section here: https://github.com/noahgift/recommendations/blob/master/notebooks/NLP_GCP.ipynb

In [77]:
# test that the authentication works
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [78]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [79]:
text = "LeBron James plays for the Cleveland Cavaliers."
client = language.LanguageServiceClient()
document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)
entities = client.analyze_entities(document).entities

In [80]:
entities

[name: "LeBron James"
type: PERSON
metadata {
  key: "mid"
  value: "/m/01jz6d"
}
metadata {
  key: "wikipedia_url"
  value: "https://en.wikipedia.org/wiki/LeBron_James"
}
salience: 0.8991044759750366
mentions {
  text {
    content: "LeBron James"
    begin_offset: -1
  }
  type: PROPER
}
, name: "Cleveland Cavaliers"
type: ORGANIZATION
metadata {
  key: "mid"
  value: "/m/0jm7n"
}
metadata {
  key: "wikipedia_url"
  value: "https://en.wikipedia.org/wiki/Cleveland_Cavaliers"
}
salience: 0.10089549422264099
mentions {
  text {
    content: "Cleveland Cavaliers"
    begin_offset: -1
  }
  type: PROPER
}
]

In [81]:
import six

In [82]:
# Sentiment analysis
client = language.LanguageServiceClient()

def sentiment_text(text):
    """Detects sentiment in the text."""
    
    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects sentiment in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    sentiment = client.analyze_sentiment(document).document_sentiment

    return(sentiment.score)
    #print('Magnitude: {}'.format(sentiment.magnitude))
    
def magnitude_text(text):
    """Detects magnitude (emotion) in the text."""
    
    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    sentiment = client.analyze_sentiment(document).document_sentiment

    return(sentiment.magnitude)

In [83]:
# Test it out on a simple positive sentence
sentiment_text("Scott loves his machine learning class")

0.800000011920929

In [84]:
# and on a negative sentence
sentiment_text("Scott hates his machine learning class")

0.30000001192092896

In [25]:
urban_subset = urban.sample(50)

In [26]:
# run it on a subset to make sure it works
urban_subset["Sentiment"] = urban_subset["definition"].apply(sentiment_text)

In [82]:
urban_subset.sample(5)

,word_id,word,up_votes,down_votes,author,definition,total_votes,Sentiment
1996010,5690860.0,The Gap,120,45,c9c9fec9,The place of a girls vagina in between her leg...,165,-0.1
1332765,3690587.0,babble babble,7,1,569b22ac,usually a word used to make noises and past ti...,8,-0.5
2389335,7432418.0,russian snowmobile,1,2,a84d5765,Its is when 2 guys are having sex and he stick...,3,-0.2
216378,523090.0,on the jazz,42,8,61df158d,A term used by [Mr T] With varied meanings;;;;...,50,0.3
169606,379137.0,coprolites,2,6,1f9dce3,Old crap that was farted out 61 million years ...,8,-0.9


In [90]:
# Apply sentiment and magnitude detection to the subsets created earlier for the ratios
best_ratio["Google_Sentiment"] = best_ratio["definition"].apply(sentiment_text)
best_ratio["Google_Magnitude"] = best_ratio["definition"].apply(magnitude_text)

worst_ratio["Google_Sentiment"] = worst_ratio["definition"].apply(sentiment_text)
worst_ratio["Google_Magnitude"] = worst_ratio["definition"].apply(magnitude_text)

In [91]:
best_ratio

,word_id,word,up_votes,down_votes,author,definition,total_votes,ratio,Google_Sentiment,Google_Magnitude
2325157,7070316.0,Taffanaut,1585,0,b82a7b78,Person born in Wales (UK) who has travelled in...,1585,inf,-0.2,0.2
2429196,7731747.0,yofam,2067,0,ae5f5426,Yofam is a greeting from one person to another...,2067,inf,0.0,0.0
2354713,7219861.0,Thanksgivukkah,3860,2,69586308,Thanksgivukkah is the name given to the conver...,3862,1930.000000,0.2,2.1
2426255,7705693.0,seachellefly,4140,6,da0fa52a,"A cheating, conniving, lying person of the fem...",4146,690.000000,-0.9,0.9
2464261,7905086.0,Wisdom Bomb,1106,2,e8fceb4e,Sage advice given.,1108,553.000000,0.0,0.0
2490678,8033884.0,rateboost,1028,2,70d8e7c4,To dishonestly boost your rates for personal g...,1030,514.000000,0.0,0.3
2228315,6466831.0,transpants,4408,9,e8fceb4e,Pants designed for the female figure that add ...,4417,489.777778,-0.3,0.3
2389282,7431915.0,lady giblets,1773,4,e8fceb4e,Female reproductive organs.,1777,443.250000,0.1,0.1
2286211,6851605.0,SigEp Saddle,1277,3,72639e78,The name given to a sorostitute/slam-piece tha...,1280,425.666667,-0.1,1.1
2236827,6523029.0,Social Securitits,1583,4,3682453e,Women over the age of 55 who refuse to wear a ...,1587,395.750000,-0.5,1.0


In [92]:
worst_ratio

,word_id,word,up_votes,down_votes,author,definition,total_votes,ratio,Google_Sentiment,Google_Magnitude
1297288,3595236.0,Chuck Norris,2,3225,2a4b26be,"The one Supreme Being, the creator and ruler o...",3227,0.000620,0.7,0.7
1011586,2758972.0,chuck norris,2,3203,cd0de555,Chuck Norris is allegedly the only man to ever...,3205,0.000624,-0.4,0.8
1600263,4415866.0,cunt,2,2905,623dffdd,I word that you should never fucking say you s...,2907,0.000688,-0.9,0.9
363848,966591.0,cudoos,3,3916,b6a386ef,basically just another word for [cool];;[wicke...,3919,0.000766,0.6,0.6
2073402,5911541.0,Chuck Norris,3,3198,244bcc15,A man who has 4 biographies written simply on ...,3201,0.000938,0.0,0.0
2143312,6124928.0,Chuck-Norris,3,3181,c746abf7,A Super Human Who Can Do Anything...!,3184,0.000943,0.6,0.6
1339648,3718026.0,H-O-L-L-A-N-D,3,3068,64807037,A old fashioned acronymn for:;;Here;;Our;;Love...,3071,0.000978,-0.2,0.2
1032912,2804865.0,shadana O'connor,8,6851,c78622,biggest fuckin slut in the world!,6859,0.001168,-0.3,0.3
2002270,5711040.0,chuck norris,4,3210,89d42301,the sum of everything awesome and gnarley,3214,0.001246,0.9,0.9
731871,1954079.0,forgoogle,2,1542,d4ca24e4,to get rid of an unpleasant fact by suppressin...,1544,0.001297,-0.5,0.5


<hr style="width:50%;">
# <a id="six">Second API</a>
<p><a href="#toc">Jump to top</a></p>

In [95]:
import boto3

In [96]:
comprehend = boto3.client(service_name='comprehend')

NoRegionError: You must specify a region.

In [ ]:

text = "It is raining today in Seattle"
print('Calling DetectSentiment')
print(json.dumps(comprehend.detect_sentiment(Text=text, LanguageCode='en'), sort_keys=True, indent=4))
print('End of DetectSentiment\n')